In [9]:
import librosa
import librosa.display
import pandas as pd
%matplotlib inline
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
tqdm.pandas()
from matplotlib import rcParams
rcParams['axes.titlepad'] = 20
import os
from os import path
import pickle
import math

#Clustering
from sklearn.cluster import DBSCAN

# Play with Librosa

In [ ]:
directory = "/Users/ydaihes/Google Drive/datahack2019/taylor/"
sub_dirs = [x[0] for x in os.walk(directory)]
song_files = []
for sub_dir in sub_dirs:
    for r, d, f in os.walk(sub_dir):
        for file in f:
            if '.mp3' in file:
                song_files.append(os.path.join(r, file))
                
song_files = set(song_files)

In [ ]:
size_of_window = 10 #in seconds

songs_data = {}
only_windows = []
for ind,song in enumerate(tqdm(song_files)):
    
    songs_data[ind] = {}
    songs_data[ind]["path"] = song
    
    y, sr = librosa.load(song)
    # trim silent edges
    trimmed_song, _ = librosa.effects.trim(y)
    
    n_fft = 2048
    hop_length = 512
    n_mels = 128
    S = librosa.feature.melspectrogram(trimmed_song, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)
    S_DB = librosa.power_to_db(S, ref=np.max)
    
    windows_of_melspec_idx = []
    melspec_data_of_windows = []
    for i in range(int(librosa.core.get_duration(trimmed_song)//size_of_window)):
        windows_of_melspec_idx.append(slice(*librosa.time_to_frames([i*size_of_window, i*size_of_window+ size_of_window], sr=sr, hop_length=hop_length)))
    #windows_of_melspec_idx.append(slice(librosa.time_to_frames(i*10+ 10),S_DB.shape[1])) # add last window even if shorter than 10 secs, will need padding
    for slice_idx in windows_of_melspec_idx:
        melspec_data_of_windows.append(S_DB[:,slice_idx][:,:math.floor((22050/512)*size_of_window)])
    print(len(melspec_data_of_windows))
    only_windows.extend(melspec_data_of_windows)
    songs_data[ind]["melspec_data_of_windows"] = melspec_data_of_windows
    #if ind == 10:
    #    break
    
with open('only_windows.pickle', 'wb') as handle:
    pickle.dump(only_windows, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [11]:
with open('only_windows.pickle', 'rb') as handle:
    only_windows = pickle.load(handle)


# Clustering

In [12]:
nsongs, nwindows, size_window = np.array(only_windows).shape
d2_train_dataset = np.array(only_windows).reshape((nsongs*nwindows,size_window))

In [20]:
hyper_params = {
    "epsilon" : 100,
    "min_samples" : 2,
    #"min_cluster_num" : 
}

db = DBSCAN(eps = hyper_params["epsilon"], min_samples = hyper_params["min_samples"], metric='euclidean',algorithm='brute').fit(d2_train_dataset[:40000])
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_ 
print(pd.Series(labels).value_counts())

-1       13786
 133      7516
 3304      229
 17        182
 3294      143
 1889       94
 1947       87
 400        87
 4172       85
 3511       83
 844        81
 1958       80
 2125       79
 3438       78
 318        77
 408        77
 3481       76
 638        76
 308        74
 3235       72
 3224       72
 3051       70
 3247       70
 3314       70
 2052       68
 1988       68
 2607       68
 3454       67
 3465       66
 274        66
         ...  
 2133        2
 2581        2
 2357        2
 532         2
 2549        2
 500         2
 2517        2
 468         2
 2485        2
 436         2
 2453        2
 404         2
 2421        2
 372         2
 340         2
 2325        2
 4182        2
 276         2
 244         2
 2261        2
 212         2
 4278        2
 2229        2
 180         2
 4246        2
 148         2
 4214        2
 2165        2
 116         2
 0           2
Length: 4300, dtype: int64
